In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 128
tree_depth = 10
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.14717960357666 | KNN Loss: 6.230565071105957 | BCE Loss: 1.916614294052124
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.178574562072754 | KNN Loss: 6.230663299560547 | BCE Loss: 1.947911262512207
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.182377815246582 | KNN Loss: 6.230414390563965 | BCE Loss: 1.9519634246826172
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.153038024902344 | KNN Loss: 6.230363368988037 | BCE Loss: 1.9226750135421753
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.175795555114746 | KNN Loss: 6.23012638092041 | BCE Loss: 1.945669174194336
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.15682315826416 | KNN Loss: 6.229822635650635 | BCE Loss: 1.9270007610321045
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.178810119628906 | KNN Loss: 6.229875564575195 | BCE Loss: 1.9489349126815796
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.180387496948242 | KNN Loss: 6.229690074920654 | BCE Loss: 1.95069742202

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.28761625289917 | KNN Loss: 6.1548004150390625 | BCE Loss: 1.1328157186508179
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.281442642211914 | KNN Loss: 6.142515659332275 | BCE Loss: 1.1389267444610596
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 7.249922275543213 | KNN Loss: 6.137760639190674 | BCE Loss: 1.1121617555618286
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 7.2522125244140625 | KNN Loss: 6.126208305358887 | BCE Loss: 1.1260042190551758
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 7.251084327697754 | KNN Loss: 6.1104416847229 | BCE Loss: 1.1406424045562744
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 7.217717170715332 | KNN Loss: 6.0983428955078125 | BCE Loss: 1.11937415599823
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 7.160482406616211 | KNN Loss: 6.068140983581543 | BCE Loss: 1.092341423034668
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 7.165571212768555 | KNN Loss: 6.057934284210205 | BCE Loss: 1

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 6.108476638793945 | KNN Loss: 5.061213493347168 | BCE Loss: 1.0472630262374878
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 6.104894161224365 | KNN Loss: 5.063118934631348 | BCE Loss: 1.0417752265930176
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 6.156625747680664 | KNN Loss: 5.079104900360107 | BCE Loss: 1.0775208473205566
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 6.1539106369018555 | KNN Loss: 5.085153102874756 | BCE Loss: 1.0687572956085205
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 6.137800216674805 | KNN Loss: 5.079092502593994 | BCE Loss: 1.058707594871521
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 6.13166618347168 | KNN Loss: 5.064792633056641 | BCE Loss: 1.066873550415039
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 6.091497421264648 | KNN Loss: 5.0570597648620605 | BCE Loss: 1.034437656402588
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 6.151091575622559 | KNN Loss: 5.051975250244141 | BCE Loss:

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 6.107427597045898 | KNN Loss: 5.051238536834717 | BCE Loss: 1.0561891794204712
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 6.115278720855713 | KNN Loss: 5.04813814163208 | BCE Loss: 1.0671404600143433
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 6.083350658416748 | KNN Loss: 5.041615009307861 | BCE Loss: 1.0417355298995972
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 6.092284202575684 | KNN Loss: 5.055273532867432 | BCE Loss: 1.037010669708252
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 6.086962699890137 | KNN Loss: 5.048601150512695 | BCE Loss: 1.0383614301681519
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 6.096273422241211 | KNN Loss: 5.04991340637207 | BCE Loss: 1.0463597774505615
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 6.074934959411621 | KNN Loss: 5.046614646911621 | BCE Loss: 1.0283203125
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 6.096368789672852 | KNN Loss: 5.045067310333252 | BCE Loss: 1.0513

Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 6.062371730804443 | KNN Loss: 5.03947639465332 | BCE Loss: 1.0228954553604126
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 6.0904436111450195 | KNN Loss: 5.043878555297852 | BCE Loss: 1.0465651750564575
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 6.092868804931641 | KNN Loss: 5.044665336608887 | BCE Loss: 1.048203468322754
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 6.085016250610352 | KNN Loss: 5.041781425476074 | BCE Loss: 1.0432350635528564
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 6.1180877685546875 | KNN Loss: 5.065965175628662 | BCE Loss: 1.0521225929260254
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 6.081908226013184 | KNN Loss: 5.034112930297852 | BCE Loss: 1.0477955341339111
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 6.106303691864014 | KNN Loss: 5.046926498413086 | BCE Loss: 1.0593771934509277
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 6.1107258796691895 | KNN Loss: 5.0440263748168945 | BCE Lo

...

Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 6.085725784301758 | KNN Loss: 5.032927989959717 | BCE Loss: 1.052797555923462
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 6.07209587097168 | KNN Loss: 5.041560173034668 | BCE Loss: 1.0305359363555908
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 6.062740802764893 | KNN Loss: 5.03169059753418 | BCE Loss: 1.031050205230713
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 6.059811592102051 | KNN Loss: 5.02908182144165 | BCE Loss: 1.0307297706604004
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 6.049190521240234 | KNN Loss: 5.028652191162109 | BCE Loss: 1.020538091659546
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 6.059130668640137 | KNN Loss: 5.042571067810059 | BCE Loss: 1.016559362411499
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 6.058109283447266 | KNN Loss: 5.032116413116455 | BCE Loss: 1.0259931087493896
Epoch   450: reducing learning rate of group 0 to 2.7058e-08.
Epoch 450 / 500 | iteration 0 / 30 | Total L

Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 6.046849727630615 | KNN Loss: 5.028506278991699 | BCE Loss: 1.0183435678482056
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 6.051676273345947 | KNN Loss: 5.032701015472412 | BCE Loss: 1.0189752578735352
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 6.064789295196533 | KNN Loss: 5.031075954437256 | BCE Loss: 1.0337133407592773
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 6.062212944030762 | KNN Loss: 5.031175136566162 | BCE Loss: 1.0310375690460205
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 6.075439453125 | KNN Loss: 5.028006553649902 | BCE Loss: 1.0474330186843872
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 6.065629959106445 | KNN Loss: 5.045800685882568 | BCE Loss: 1.0198293924331665
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 6.087003707885742 | KNN Loss: 5.031047344207764 | BCE Loss: 1.0559563636779785
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 6.056163787841797 | KNN Loss: 5.03107213973999 | BCE

Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 6.060054779052734 | KNN Loss: 5.034311771392822 | BCE Loss: 1.0257431268692017
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 6.076012134552002 | KNN Loss: 5.031058311462402 | BCE Loss: 1.0449538230895996
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 6.061872482299805 | KNN Loss: 5.039194107055664 | BCE Loss: 1.0226786136627197
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 6.129312992095947 | KNN Loss: 5.029121398925781 | BCE Loss: 1.1001917123794556
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 6.044210433959961 | KNN Loss: 5.0272393226623535 | BCE Loss: 1.0169711112976074
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 6.068950176239014 | KNN Loss: 5.03914213180542 | BCE Loss: 1.0298079252243042
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 6.090576171875 | KNN Loss: 5.035552501678467 | BCE Loss: 1.0550237894058228
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 6.102415084838867 | KNN Loss: 5.051633358001709 | BCE

Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 6.085815906524658 | KNN Loss: 5.036717414855957 | BCE Loss: 1.0490984916687012
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 6.078423500061035 | KNN Loss: 5.040165901184082 | BCE Loss: 1.0382577180862427
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 6.091352462768555 | KNN Loss: 5.043668746948242 | BCE Loss: 1.0476837158203125
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 6.054566860198975 | KNN Loss: 5.036487579345703 | BCE Loss: 1.0180792808532715
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 6.045516490936279 | KNN Loss: 5.042888641357422 | BCE Loss: 1.0026277303695679
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 6.068411827087402 | KNN Loss: 5.026695251464844 | BCE Loss: 1.041716456413269
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 6.059846878051758 | KNN Loss: 5.0303053855896 | BCE Loss: 1.029541254043579
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 6.07157564163208 | KNN Loss: 5.039799690246582 | BCE L

Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 6.06508731842041 | KNN Loss: 5.025021076202393 | BCE Loss: 1.0400664806365967
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 6.080235004425049 | KNN Loss: 5.030549049377441 | BCE Loss: 1.0496859550476074
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 6.075401782989502 | KNN Loss: 5.027512550354004 | BCE Loss: 1.0478891134262085
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 6.110662460327148 | KNN Loss: 5.062050819396973 | BCE Loss: 1.0486117601394653
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 6.048522472381592 | KNN Loss: 5.0362067222595215 | BCE Loss: 1.0123156309127808
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 6.075016021728516 | KNN Loss: 5.038321495056152 | BCE Loss: 1.0366945266723633
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 6.092489719390869 | KNN Loss: 5.0410990715026855 | BCE Loss: 1.051390528678894
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 6.068001747131348 | KNN Loss: 5.043178558349609 |

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 3.1004,  2.6817,  2.7539,  2.4920,  3.6779,  0.7150,  2.7538,  2.4548,
          2.5670,  1.9031,  2.4237,  2.1422,  0.9989,  1.9468,  1.3392,  1.4412,
          1.9644,  2.2851,  2.9922,  2.3174,  1.7563,  3.0557,  2.2364,  1.6399,
          2.4427,  1.5676,  2.3108,  1.6814,  1.3940,  0.0944, -0.1598,  1.1897,
          0.2207,  1.0250,  1.5466,  1.4039,  0.7653,  3.5941,  0.6789,  1.5340,
          0.9580, -0.7711, -0.3380,  2.2785,  2.4960,  0.7685, -0.0435, -0.1210,
          1.4144,  2.7551,  1.6707,  0.1121,  1.4718,  0.4221, -0.7428,  1.1006,
          1.6403,  1.3968,  1.3351,  1.9359,  0.2931,  0.7008,  0.2174,  1.9189,
          1.3878,  1.7041, -1.8440,  0.3478,  2.1132,  1.7647,  2.5614,  0.4807,
          1.2871,  2.6378,  1.7086,  1.1656,  0.2173,  0.7676,  0.3412,  1.7553,
          0.0473,  0.2614,  2.0176, -0.3224,  0.4090, -1.1499, -2.7509, -0.2275,
          0.5427, -1.7010,  0.4813, -0.1728, -0.4958, -0.8848,  0.5925,  1.2779,
         -0.5448, -0.9115,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 84.87it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 029 | Total loss: 9.606 | Reg loss: 0.012 | Tree loss: 9.606 | Accuracy: 0.000000 | 1.466 sec/iter
Epoch: 00 | Batch: 001 / 029 | Total loss: 9.597 | Reg loss: 0.011 | Tree loss: 9.597 | Accuracy: 0.000000 | 1.161 sec/iter
Epoch: 00 | Batch: 002 / 029 | Total loss: 9.588 | Reg loss: 0.010 | Tree loss: 9.588 | Accuracy: 0.000000 | 1.091 sec/iter
Epoch: 00 | Batch: 003 / 029 | Total loss: 9.579 | Reg loss: 0.010 | Tree loss: 9.579 | Accuracy: 0.000000 | 1.04 sec/iter
Epoch: 00 | Batch: 004 / 029 | Total loss: 9.572 | Reg loss: 0.009 | Tree loss: 9.572 | Accuracy: 0.000000 | 1.011 sec/iter
Epoch: 00 | Batch: 005 / 029 | Total loss: 9.562 | Reg loss: 0.009 | Tree loss: 9.562 | Accuracy: 0.003906 | 0.989 sec/iter
Epoch: 00 | Batch: 006 / 029 | Total loss: 9.552 | Reg loss: 0.009 | Tree loss: 9.552 | Accuracy: 0.005859 | 0.975 s

Epoch: 02 | Batch: 003 / 029 | Total loss: 9.348 | Reg loss: 0.009 | Tree loss: 9.348 | Accuracy: 0.390625 | 0.918 sec/iter
Epoch: 02 | Batch: 004 / 029 | Total loss: 9.345 | Reg loss: 0.009 | Tree loss: 9.345 | Accuracy: 0.332031 | 0.917 sec/iter
Epoch: 02 | Batch: 005 / 029 | Total loss: 9.330 | Reg loss: 0.009 | Tree loss: 9.330 | Accuracy: 0.388672 | 0.917 sec/iter
Epoch: 02 | Batch: 006 / 029 | Total loss: 9.338 | Reg loss: 0.010 | Tree loss: 9.338 | Accuracy: 0.285156 | 0.916 sec/iter
Epoch: 02 | Batch: 007 / 029 | Total loss: 9.321 | Reg loss: 0.010 | Tree loss: 9.321 | Accuracy: 0.376953 | 0.916 sec/iter
Epoch: 02 | Batch: 008 / 029 | Total loss: 9.313 | Reg loss: 0.010 | Tree loss: 9.313 | Accuracy: 0.363281 | 0.916 sec/iter
Epoch: 02 | Batch: 009 / 029 | Total loss: 9.297 | Reg loss: 0.011 | Tree loss: 9.297 | Accuracy: 0.367188 | 0.916 sec/iter
Epoch: 02 | Batch: 010 / 029 | Total loss: 9.291 | Reg loss: 0.011 | Tree loss: 9.291 | Accuracy: 0.312500 | 0.916 sec/iter
Epoch: 0

Epoch: 04 | Batch: 007 / 029 | Total loss: 9.029 | Reg loss: 0.016 | Tree loss: 9.029 | Accuracy: 0.376953 | 0.901 sec/iter
Epoch: 04 | Batch: 008 / 029 | Total loss: 9.031 | Reg loss: 0.017 | Tree loss: 9.031 | Accuracy: 0.363281 | 0.901 sec/iter
Epoch: 04 | Batch: 009 / 029 | Total loss: 9.035 | Reg loss: 0.017 | Tree loss: 9.035 | Accuracy: 0.314453 | 0.901 sec/iter
Epoch: 04 | Batch: 010 / 029 | Total loss: 9.004 | Reg loss: 0.017 | Tree loss: 9.004 | Accuracy: 0.330078 | 0.9 sec/iter
Epoch: 04 | Batch: 011 / 029 | Total loss: 8.990 | Reg loss: 0.018 | Tree loss: 8.990 | Accuracy: 0.355469 | 0.9 sec/iter
Epoch: 04 | Batch: 012 / 029 | Total loss: 8.961 | Reg loss: 0.018 | Tree loss: 8.961 | Accuracy: 0.355469 | 0.9 sec/iter
Epoch: 04 | Batch: 013 / 029 | Total loss: 8.950 | Reg loss: 0.019 | Tree loss: 8.950 | Accuracy: 0.355469 | 0.9 sec/iter
Epoch: 04 | Batch: 014 / 029 | Total loss: 8.946 | Reg loss: 0.019 | Tree loss: 8.946 | Accuracy: 0.367188 | 0.9 sec/iter
Epoch: 04 | Batch:

Epoch: 06 | Batch: 011 / 029 | Total loss: 8.561 | Reg loss: 0.023 | Tree loss: 8.561 | Accuracy: 0.380859 | 0.898 sec/iter
Epoch: 06 | Batch: 012 / 029 | Total loss: 8.541 | Reg loss: 0.024 | Tree loss: 8.541 | Accuracy: 0.347656 | 0.898 sec/iter
Epoch: 06 | Batch: 013 / 029 | Total loss: 8.520 | Reg loss: 0.024 | Tree loss: 8.520 | Accuracy: 0.339844 | 0.898 sec/iter
Epoch: 06 | Batch: 014 / 029 | Total loss: 8.498 | Reg loss: 0.024 | Tree loss: 8.498 | Accuracy: 0.333984 | 0.898 sec/iter
Epoch: 06 | Batch: 015 / 029 | Total loss: 8.469 | Reg loss: 0.025 | Tree loss: 8.469 | Accuracy: 0.343750 | 0.898 sec/iter
Epoch: 06 | Batch: 016 / 029 | Total loss: 8.454 | Reg loss: 0.025 | Tree loss: 8.454 | Accuracy: 0.353516 | 0.898 sec/iter
Epoch: 06 | Batch: 017 / 029 | Total loss: 8.435 | Reg loss: 0.026 | Tree loss: 8.435 | Accuracy: 0.345703 | 0.898 sec/iter
Epoch: 06 | Batch: 018 / 029 | Total loss: 8.432 | Reg loss: 0.026 | Tree loss: 8.432 | Accuracy: 0.365234 | 0.898 sec/iter
Epoch: 0

Epoch: 08 | Batch: 015 / 029 | Total loss: 7.950 | Reg loss: 0.028 | Tree loss: 7.950 | Accuracy: 0.363281 | 0.897 sec/iter
Epoch: 08 | Batch: 016 / 029 | Total loss: 7.949 | Reg loss: 0.028 | Tree loss: 7.949 | Accuracy: 0.371094 | 0.897 sec/iter
Epoch: 08 | Batch: 017 / 029 | Total loss: 7.916 | Reg loss: 0.028 | Tree loss: 7.916 | Accuracy: 0.365234 | 0.897 sec/iter
Epoch: 08 | Batch: 018 / 029 | Total loss: 7.935 | Reg loss: 0.029 | Tree loss: 7.935 | Accuracy: 0.332031 | 0.897 sec/iter
Epoch: 08 | Batch: 019 / 029 | Total loss: 7.903 | Reg loss: 0.029 | Tree loss: 7.903 | Accuracy: 0.361328 | 0.897 sec/iter
Epoch: 08 | Batch: 020 / 029 | Total loss: 7.874 | Reg loss: 0.029 | Tree loss: 7.874 | Accuracy: 0.320312 | 0.897 sec/iter
Epoch: 08 | Batch: 021 / 029 | Total loss: 7.850 | Reg loss: 0.029 | Tree loss: 7.850 | Accuracy: 0.376953 | 0.897 sec/iter
Epoch: 08 | Batch: 022 / 029 | Total loss: 7.799 | Reg loss: 0.030 | Tree loss: 7.799 | Accuracy: 0.341797 | 0.896 sec/iter
Epoch: 0

...

Epoch: 89 | Batch: 013 / 029 | Total loss: 1.607 | Reg loss: 0.038 | Tree loss: 1.607 | Accuracy: 0.396484 | 0.894 sec/iter
Epoch: 89 | Batch: 014 / 029 | Total loss: 1.585 | Reg loss: 0.038 | Tree loss: 1.585 | Accuracy: 0.357422 | 0.894 sec/iter
Epoch: 89 | Batch: 015 / 029 | Total loss: 1.535 | Reg loss: 0.038 | Tree loss: 1.535 | Accuracy: 0.382812 | 0.894 sec/iter
Epoch: 89 | Batch: 016 / 029 | Total loss: 1.552 | Reg loss: 0.038 | Tree loss: 1.552 | Accuracy: 0.347656 | 0.894 sec/iter
Epoch: 89 | Batch: 017 / 029 | Total loss: 1.480 | Reg loss: 0.039 | Tree loss: 1.480 | Accuracy: 0.378906 | 0.894 sec/iter
Epoch: 89 | Batch: 018 / 029 | Total loss: 1.470 | Reg loss: 0.039 | Tree loss: 1.470 | Accuracy: 0.380859 | 0.894 sec/iter
Epoch: 89 | Batch: 019 / 029 | Total loss: 1.507 | Reg loss: 0.039 | Tree loss: 1.507 | Accuracy: 0.365234 | 0.894 sec/iter
Epoch: 89 | Batch: 020 / 029 | Total loss: 1.495 | Reg loss: 0.039 | Tree loss: 1.495 | Accuracy: 0.365234 | 0.894 sec/iter
Epoch: 8

Epoch: 91 | Batch: 017 / 029 | Total loss: 1.525 | Reg loss: 0.038 | Tree loss: 1.525 | Accuracy: 0.382812 | 0.894 sec/iter
Epoch: 91 | Batch: 018 / 029 | Total loss: 1.547 | Reg loss: 0.038 | Tree loss: 1.547 | Accuracy: 0.373047 | 0.894 sec/iter
Epoch: 91 | Batch: 019 / 029 | Total loss: 1.509 | Reg loss: 0.039 | Tree loss: 1.509 | Accuracy: 0.349609 | 0.894 sec/iter
Epoch: 91 | Batch: 020 / 029 | Total loss: 1.510 | Reg loss: 0.039 | Tree loss: 1.510 | Accuracy: 0.396484 | 0.894 sec/iter
Epoch: 91 | Batch: 021 / 029 | Total loss: 1.545 | Reg loss: 0.039 | Tree loss: 1.545 | Accuracy: 0.335938 | 0.894 sec/iter
Epoch: 91 | Batch: 022 / 029 | Total loss: 1.469 | Reg loss: 0.039 | Tree loss: 1.469 | Accuracy: 0.378906 | 0.894 sec/iter
Epoch: 91 | Batch: 023 / 029 | Total loss: 1.493 | Reg loss: 0.039 | Tree loss: 1.493 | Accuracy: 0.365234 | 0.894 sec/iter
Epoch: 91 | Batch: 024 / 029 | Total loss: 1.492 | Reg loss: 0.039 | Tree loss: 1.492 | Accuracy: 0.353516 | 0.894 sec/iter
Epoch: 9

Epoch: 93 | Batch: 021 / 029 | Total loss: 1.472 | Reg loss: 0.039 | Tree loss: 1.472 | Accuracy: 0.373047 | 0.894 sec/iter
Epoch: 93 | Batch: 022 / 029 | Total loss: 1.484 | Reg loss: 0.039 | Tree loss: 1.484 | Accuracy: 0.359375 | 0.894 sec/iter
Epoch: 93 | Batch: 023 / 029 | Total loss: 1.472 | Reg loss: 0.039 | Tree loss: 1.472 | Accuracy: 0.355469 | 0.894 sec/iter
Epoch: 93 | Batch: 024 / 029 | Total loss: 1.501 | Reg loss: 0.039 | Tree loss: 1.501 | Accuracy: 0.371094 | 0.894 sec/iter
Epoch: 93 | Batch: 025 / 029 | Total loss: 1.474 | Reg loss: 0.039 | Tree loss: 1.474 | Accuracy: 0.341797 | 0.894 sec/iter
Epoch: 93 | Batch: 026 / 029 | Total loss: 1.494 | Reg loss: 0.039 | Tree loss: 1.494 | Accuracy: 0.388672 | 0.894 sec/iter
Epoch: 93 | Batch: 027 / 029 | Total loss: 1.477 | Reg loss: 0.039 | Tree loss: 1.477 | Accuracy: 0.375000 | 0.894 sec/iter
Epoch: 93 | Batch: 028 / 029 | Total loss: 1.467 | Reg loss: 0.039 | Tree loss: 1.467 | Accuracy: 0.370370 | 0.894 sec/iter
Average 

Epoch: 95 | Batch: 025 / 029 | Total loss: 1.457 | Reg loss: 0.039 | Tree loss: 1.457 | Accuracy: 0.365234 | 0.894 sec/iter
Epoch: 95 | Batch: 026 / 029 | Total loss: 1.423 | Reg loss: 0.039 | Tree loss: 1.423 | Accuracy: 0.357422 | 0.894 sec/iter
Epoch: 95 | Batch: 027 / 029 | Total loss: 1.479 | Reg loss: 0.039 | Tree loss: 1.479 | Accuracy: 0.349609 | 0.894 sec/iter
Epoch: 95 | Batch: 028 / 029 | Total loss: 1.424 | Reg loss: 0.039 | Tree loss: 1.424 | Accuracy: 0.387205 | 0.894 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 96 | Batch: 000 / 029 | Total loss: 1.843 | Reg loss: 0.037 | Tree loss: 1.843 | Accuracy: 0.363281 | 0.894 sec/iter
Epoch: 96 | Batch: 001 / 029 | Total loss: 1.832 | Reg loss: 0.037 | Tree loss: 1.832 | Ac

Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 98 | Batch: 000 / 029 | Total loss: 1.793 | Reg loss: 0.037 | Tree loss: 1.793 | Accuracy: 0.355469 | 0.894 sec/iter
Epoch: 98 | Batch: 001 / 029 | Total loss: 1.799 | Reg loss: 0.037 | Tree loss: 1.799 | Accuracy: 0.363281 | 0.894 sec/iter
Epoch: 98 | Batch: 002 / 029 | Total loss: 1.765 | Reg loss: 0.037 | Tree loss: 1.765 | Accuracy: 0.363281 | 0.894 sec/iter
Epoch: 98 | Batch: 003 / 029 | Total loss: 1.777 | Reg loss: 0.037 | Tree loss: 1.777 | Accuracy: 0.365234 | 0.894 sec/iter
Epoch: 98 | Batch: 004 / 029 | Total loss: 1.752 | Reg loss: 0.037 | Tree loss: 1.752 | Accuracy: 0.375000 | 0.894 sec/iter
Epoch: 98 | Batch: 005 / 029 | Total loss: 1.695 | Reg loss: 0.037 | Tree loss: 1.695 | Ac

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 9.652777777777779


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 720


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 62 ==============
============== Pattern 63 ==============
============== Pattern 64 ==============
============== Pattern 65 ==============
============== Pattern 66 ==============
============== Pattern 67 ==============
============== Pattern 68 ==============
============== Pattern 69 ==============
============== Pattern 70 ==============
============== Pattern 71 ==============
============== Pattern 72 ==============
============== Pattern 73 ==============
============== Pattern 74 ==============
============== Pattern 75 ==============
============== Pattern 76 ==============
============== Pattern 77 ==============
============== Pattern 78 ==============
============== Pattern 79 ==============
============== Pattern 80 ==============
============== Pattern 81 ==============
============== Pattern 82 ==============
============== Pattern 83 ==============
14633
============== Pattern 84 ==============
============== Pattern 85 ==============
==========

============== Pattern 609 ==============
============== Pattern 610 ==============
============== Pattern 611 ==============
============== Pattern 612 ==============
============== Pattern 613 ==============
============== Pattern 614 ==============
============== Pattern 615 ==============
============== Pattern 616 ==============
============== Pattern 617 ==============
============== Pattern 618 ==============
============== Pattern 619 ==============
============== Pattern 620 ==============
============== Pattern 621 ==============
============== Pattern 622 ==============
============== Pattern 623 ==============
============== Pattern 624 ==============
============== Pattern 625 ==============
============== Pattern 626 ==============
============== Pattern 627 ==============
============== Pattern 628 ==============
============== Pattern 629 ==============
============== Pattern 630 ==============
============== Pattern 631 ==============
============== Pattern 632 =======